In [1]:
import requests
from requests.adapters import HTTPAdapter, Retry
from pprint import pprint
import time
import os
from git import Repo
import pandas as pd
GITHUB_TOKEN = ""

In [2]:
def get_url(low_star, high_star, page):
    return f'https://api.github.com/search/repositories?q=stars:{low_star}..{high_star}&sort=stars&order=desc&per_page=50&page={page}'

In [3]:
page_url = get_url(3000,3100,1)
response = requests.get(page_url , auth=('dzheng', GITHUB_TOKEN))
print(f"Status code: ", response.status_code)

Status code:  200


In [4]:
def get_last_page(response):
    if 'Link' not in response.headers:
        return 1
    last_page = response.headers['Link'][-len('; rel="last"') - 10 :-len('; rel="last"')]
    last_page = last_page[last_page.find('page')+len('page')+1:]
    last_page = last_page[:last_page.find('>')]
    last_page = int(last_page)
    return last_page

In [5]:
get_last_page(response)

9

In [6]:
range_list = list(range(3000, 20000, 100))
print(len(range_list))# use small star range to gurantee we get all repos, less than 1k repos >20000 stars

170


In [7]:
s = requests.Session()
retries = Retry(total = 10, backoff_factor=1, status_forcelist=[403])
s.mount("https://", HTTPAdapter(max_retries=retries))

response_dfs = []
range_gap = 99
for start_range in range_list:
    print(f'Star range: {start_range} to {start_range + range_gap}')
    base_url = get_url(start_range, start_range + range_gap, 1)
    response = s.get(base_url , auth=('dzheng', GITHUB_TOKEN))
    last_page = 1
    while response.status_code != 200:
        print('Retrying')
        response = s.get(base_url , auth=('dzheng', GITHUB_TOKEN))
        time.sleep(5)
    if len(response.json()['items']) == 50:
        last_page = get_last_page(response)
    print(f'Pages {last_page}')
    for i in range(last_page):
        print('Current Page', i+1)
        cur_url = get_url(start_range, start_range + range_gap, i+1)
        cur_response = s.get(base_url , auth=('dzheng', GITHUB_TOKEN))
        while cur_response.status_code != 200:
            print('Retrying')
            cur_response = s.get(base_url , auth=('dzheng', GITHUB_TOKEN))
        time.sleep(5)
        cur_response_df = pd.DataFrame(response.json()['items'])
        response_dfs.append(cur_response_df)
        time.sleep(2)
    print()

# > 20k stars  
print('Star range: >20000') 
base_url = f'https://api.github.com/search/repositories?q=stars:>20000&sort=stars&order=desc&per_page=50&page='
response = s.get(base_url , auth=('dzheng', GITHUB_TOKEN))
last_page = get_last_page(response)
print(f'Pages {last_page}')
for i in range(last_page):
    print('Current Page', i+1)
    cur_url = f'{base_url}{i+1}'
    cur_response = requests.get(base_url , auth=('dzheng', GITHUB_TOKEN))
    cur_response_df = pd.DataFrame(response.json()['items'])
    response_dfs.append(cur_response_df)
    time.sleep(2)


Star range: 3000 to 3099
Pages 9
Current Page 1
Current Page 2
Current Page 3
Current Page 4
Current Page 5
Current Page 6
Current Page 7
Current Page 8
Current Page 9

Star range: 3100 to 3199
Pages 9
Current Page 1
Current Page 2
Current Page 3
Current Page 4
Current Page 5
Current Page 6
Current Page 7
Current Page 8
Current Page 9

Star range: 3200 to 3299
Pages 9
Current Page 1
Current Page 2
Current Page 3
Current Page 4
Current Page 5
Current Page 6
Current Page 7
Current Page 8
Current Page 9

Star range: 3300 to 3399
Pages 9
Current Page 1
Current Page 2
Current Page 3
Current Page 4
Current Page 5
Current Page 6
Current Page 7
Current Page 8
Current Page 9

Star range: 3400 to 3499
Pages 8
Current Page 1
Current Page 2
Current Page 3
Current Page 4
Current Page 5
Current Page 6
Current Page 7
Current Page 8

Star range: 3500 to 3599
Pages 7
Current Page 1
Current Page 2
Current Page 3
Current Page 4
Current Page 5
Current Page 6
Current Page 7

Star range: 3600 to 3699
Pages 

In [8]:
response_df = pd.concat(response_dfs)

In [2]:
response_df = pd.read_pickle('temp.pkl').reset_index(drop=True)

In [3]:
print(response_df.shape)
response_df.tail()

(14533, 81)


,id,node_id,name,full_name,private,owner,html_url,description,fork,url,...,is_template,web_commit_signoff_required,topics,visibility,forks,open_issues,watchers,default_branch,permissions,score
14528,576201,MDEwOlJlcG9zaXRvcnk1NzYyMDE=,three.js,mrdoob/three.js,False,"{'login': 'mrdoob', 'id': 97088, 'node_id': 'M...",https://github.com/mrdoob/three.js,JavaScript 3D Library.,False,https://api.github.com/repos/mrdoob/three.js,...,False,False,"[3d, augmented-reality, canvas, html5, javascr...",public,34214,493,89996,dev,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
14529,20929025,MDEwOlJlcG9zaXRvcnkyMDkyOTAyNQ==,TypeScript,microsoft/TypeScript,False,"{'login': 'microsoft', 'id': 6154722, 'node_id...",https://github.com/microsoft/TypeScript,TypeScript is a superset of JavaScript that co...,False,https://api.github.com/repos/microsoft/TypeScript,...,False,False,"[javascript, language, typechecker, typescript]",public,11578,6127,89580,main,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
14530,133442384,MDEwOlJlcG9zaXRvcnkxMzM0NDIzODQ=,deno,denoland/deno,False,"{'login': 'denoland', 'id': 42048915, 'node_id...",https://github.com/denoland/deno,A modern runtime for JavaScript and TypeScript.,False,https://api.github.com/repos/denoland/deno,...,False,False,"[deno, javascript, rust, typescript]",public,4799,1265,88523,main,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
14531,184456251,MDEwOlJlcG9zaXRvcnkxODQ0NTYyNTE=,PowerToys,microsoft/PowerToys,False,"{'login': 'microsoft', 'id': 6154722, 'node_id...",https://github.com/microsoft/PowerToys,Windows system utilities to maximize productivity,False,https://api.github.com/repos/microsoft/PowerToys,...,False,False,"[color-picker, desktop, fancyzones, keyboard-m...",public,5077,4704,88310,main,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
14532,100060912,MDEwOlJlcG9zaXRvcnkxMDAwNjA5MTI=,terminal,microsoft/terminal,False,"{'login': 'microsoft', 'id': 6154722, 'node_id...",https://github.com/microsoft/terminal,The new Windows Terminal and the original Wind...,False,https://api.github.com/repos/microsoft/terminal,...,False,False,"[cmd, command-line, console, terminal, windows...",public,7707,1554,88056,main,"{'admin': False, 'maintain': False, 'push': Fa...",1.0


In [4]:
response_df['has_pre_commit_hooks'] = False

In [5]:
def find_all(name, path):
    result = []
    for root, _, files in os.walk(path):
        if name in files:
            result.append(os.path.join(root, name))
    return result

In [6]:

from tqdm import tqdm
start = 2254+104+2680
# print(os.listdir())
for i in tqdm(range(start, response_df.shape[0])): 
    repo_url = response_df.iloc[i]['clone_url']
    # print(repo_url)
    os.makedirs('./cur_repo', exist_ok = True)
    os.system("rm -fr ./cur_repo") # cant be done using os.remove as we need to force
    Repo.clone_from(repo_url, './cur_repo')
    pre_com_hooks_list = find_all('.pre-commit-config.yaml', './cur_repo') # potentially too slow

    if pre_com_hooks_list:
        response_df.iloc[i,response_df.columns.get_loc('has_pre_commit_hooks')] = True
        
    for hook in pre_com_hooks_list:
        hook_folder = 'found_hooks'
        file_name = f"{repo_url[repo_url.find('github.com')+len('github.com')+1:-4].replace('/', '-')}.yaml"
        os.system(f'cp {hook} ./{hook_folder}/{file_name}')  
    # print()

  8%|▊         | 760/9495 [4:19:53<49:47:02, 20.52s/it]   


GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/raspberrypi/firmware.git ./cur_repo
  stderr: 'Cloning into './cur_repo'...
POST git-upload-pack (gzip 4627 to 2310 bytes)
error: RPC failed; curl 56 GnuTLS recv error (-54): Error in the pull function.
fatal: the remote end hung up unexpectedly
fatal: early EOF
fatal: index-pack failed
'

In [ ]:
response_df[response_df['has_pre_commit_hooks']]

,id,node_id,name,full_name,private,owner,html_url,description,fork,url,...,web_commit_signoff_required,topics,visibility,forks,open_issues,watchers,default_branch,permissions,score,has_pre_commit_hooks
18,63476337,MDEwOlJlcG9zaXRvcnk2MzQ3NjMzNw==,Python,TheAlgorithms/Python,False,"{'login': 'TheAlgorithms', 'id': 20487725, 'no...",https://github.com/TheAlgorithms/Python,All Algorithms implemented in Python,False,https://api.github.com/repos/TheAlgorithms/Python,...,False,"[algorithm, algorithm-competitions, algorithms...",public,39374,145,155621,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0,True


In [ ]:
response_df.to_pickle('3000_star.pkl')